In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
REBUILD_DATA = True #Setar True para rodar uma vez e mudar para False, a não ser que seja necessário fazer alguma alteração no dataset

#Definição da classe para o processamento de dados
class Sign_Language():
    IMG_SIZE = 50
    A = "C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_train/A"
    B = "C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_train/B"
    C = "C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_train/C"
    TESTING = "C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_test - Testrun"
    LABELS = {A: 0, B: 1, C: 2}
    training_data  = []
    
    Acount = 0
    Bcount = 0
    Ccount = 0
    
    #Função para montar os dados de treino
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        #Para caso as imagens forem de tamanhos diferentes (ou quiser redimensionar o dataset), usar a função abaixo para redimensionar para um tamanho padrão
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(3)[self.LABELS[label]]])
                        
                        #Contar a quantidade de elementos de cada classe
                        if label == self.A:
                            self.Acount += 1
                        elif label == self.B:
                            self.Bcount += 1
                        elif label == self.C:
                            self.Ccount += 1
                        
                    except Exception as e:
                        pass
                        
            
        #Sortir os dados no conjunto, para melhor resposta do modelo
        np.random.shuffle(self.training_data)
        np.save("training_data(2).npy", self.training_data)
        print('A: ', signlanguage.Acount)
        print('B: ', signlanguage.Bcount)
        print('C: ', signlanguage.Ccount)

#Se algum arquivo for alterado no dataset, executar para montar os dados de treino novamente
if REBUILD_DATA:
    signlanguage = Sign_Language()
    signlanguage.make_training_data()

C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_train/A


100%|█████████████████████████████████████| 3000/3000 [00:03<00:00, 892.78it/s]


C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_train/B


100%|█████████████████████████████████████| 3000/3000 [00:03<00:00, 792.94it/s]


C:/Users/ThiagoBS/Desktop/Trabalho de Visao Computacional/Sign Language Tests/ASL Alphabet/asl_alphabet_train/C


100%|█████████████████████████████████████| 3000/3000 [00:03<00:00, 912.92it/s]


A:  3000
B:  3000
C:  3000


In [3]:
#Para carregar o arquivo com os dados das imagens de treino
training_data = np.load("training_data(2).npy", allow_pickle = True)

#Printa a quantidade de imagens que estão carregadas na varíavel de treino
print(len(training_data))

9000


In [4]:
#Importando bibliotecas e pacotes necessários para construir o modelo de CNN
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
#Criando uma classe Net, o modelo da nossa Rede Neural

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) #input é uma imagem, output são 32 convoluções, kernel/janela de 5x5
        self.conv2 = nn.Conv2d(32, 64, 5) #input é o output da camada anterior, output são 64 convoluções...
        self.conv3 = nn.Conv2d(64, 128, 5)
        
        #É necessário passar uma imagem aplainada (flattened) para as layers fully-connected
        #Para isso criamos uma função para poder realizar esse processo
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512) #Aplainamento
        self.fc2 = nn.Linear(512, 3) #512 de entrada e 3 de saída por causa das classes (A, B e C)
        
    def convs(self, x):
        #Max Pooling 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    #Função que controla como os dados serão mandados para "frente"
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear) #Aplaina X antes
        x = F.relu(self.fc1(x))
        x = self.fc2(x) #Camada de saída, sem ativação Relu aqui
        return F.softmax(x, dim = 1)
    
    
net = Net()
print(net)      

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)


In [6]:
#Agora precisamos treinar o nosso modelo

import torch.optim as optim
#Para minimizar a taxa de perda do modelo, devemos usar um otimizador
optimizer = optim.Adam(net.parameters(), lr = 0.001) #Otimizador Adam para otimizar os parâmetros que variam, passo de aprendizado de 0.001
loss_function = nn.MSELoss()

In [7]:
#Precisamos também separar os nossos dados em X (features) e y (labels)
X = torch.Tensor([i[0] for i in training_data]).view(-1, 50, 50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

In [8]:
VAL_PCT = 0.1 #Reservando 10% dos nossos dados para validação do modelo
val_size = int(len(X)*VAL_PCT)
print(val_size)

900


In [9]:
#Definindo os X (features) e y(labels) de teste e treino
train_X = X[: -val_size]
train_y = y[: -val_size]

test_X = X[-val_size: ]
test_y = y[-val_size: ]

print(len(train_X), len(test_X))

8100 900


In [ ]:
#Nós precisamos iterar sobre os nossos dados. Para isso decidimos qual o tamanho dos batches usar.
BATCH_SIZE = 100
#EPOCHS é o número de vezes que será iterado sobre todos os dados do conjunto
EPOCHS = 3

#Definindo a função de treino
def train(net):
    for epoch in range(EPOCHS):
        for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            #Zera os gradientes ao fim de um batch
            net.zero_grad()

            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    #Atualiza o otimizador

        print(f"Epoch: {epoch}. Loss: {loss}")

train(net)

100%|██████████████████████████████████████████| 81/81 [02:20<00:00,  1.67s/it]


Epoch: 0. Loss: 0.00039744595414958894


100%|██████████████████████████████████████████| 81/81 [02:25<00:00,  1.89s/it]


Epoch: 1. Loss: 0.00031481310725212097


 64%|██████████████████████████▉               | 52/81 [01:31<00:51,  1.77s/it]

In [13]:
#Definindo a função de teste

def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i])
            net_out = net(test_X[i].view(-1, 1, 50, 50))[0] #Retorna uma lista que é o output dado pelo modelo na validação
            predicted_class = torch.argmax(net_out)
        
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Precisão: ", round(correct/total, 3))

test(net)

100%|███████████████████████████████████████| 900/900 [00:07<00:00, 116.42it/s]


Precisão:  1.0
